## Trapezoidal rule integration
**CYBR 304 & MATH 420**  <br>
_Spring 2024_ <br>

The function `Trapezoidal` returns the n-panel trapezoidal rule estimation for 
`integrate(f(x),x,a,b)`.

In [1]:
"""
    trapezoidial(f::Function, a::Number, b::Number, n::Integer)

Return the n-panel trapezoidal rule estimation for `integrate(f(x),x,a,b)`.
"""
function trapezoidal(f::Function, a::Number, b::Number, n::Integer)
    h = (b-a)/n
    s = (f(a) + f(b))/2
    for for k in 1:n-1
        s += f(fma(h,k,a))
    end
    h*s
end;

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\Barton\github\quartic\MATH-420-CYBR-304\Julia-notebooks\trap_rule.ipynb:11:8
        s += f(fma(h,k,a))
    end
#      └ ── invalid iteration spec: expected one of `=` `in` or `∈`

Let's make sure that the documentation string works correctly

In [2]:
? trapezoidal

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\Barton\github\quartic\MATH-420-CYBR-304\Julia-notebooks\trap_rule.ipynb:1:1
? trapezoidal
╙ ── not a unary operator

And let's do some basic checks. The rule is exact for every linear function; let's 
check that is the case.

trapezoidal(x -> 1, 0,1,1)

In [3]:
trapezoidal(x -> 1, 0,1,10)

┌ Error: Some Julia code in the VS Code extension crashed
└ @ VSCodeServer c:\Users\Barton\.vscode\extensions\julialang.language-julia-1.79.2\scripts\error_handler.jl:15
ERROR: UndefVarError: hasproperty not defined
Stacktrace:
 [1] (::getfield(VSCodeServer, Symbol("##19#20")){Regex})(::Base.StackTraces.StackFrame) at c:\Users\Barton\.vscode\extensions\julialang.language-julia-1.79.2\scripts\packages\VSCodeServer\src\misc.jl:37
 [2] findnext(::getfield(VSCodeServer, Symbol("##19#20")){Regex}, ::Array{Base.StackTraces.StackFrame,1}, ::Int64) at .\array.jl:1690
 [3] findfirst(::Function, ::Array{Base.StackTraces.StackFrame,1}) at .\array.jl:1741
 [4] find_first_trace_entry(::Array{Union{Ptr{Nothing}, InterpreterIP},1}, ::Regex) at c:\Users\Barton\.vscode\extensions\julialang.language-julia-1.79.2\scripts\packages\VSCodeServer\src\misc.jl:35
 [5] crop_backtrace(::Array{Union{Ptr{Nothing}, InterpreterIP},1}) at c:\Users\Barton\.vscode\extensions\julialang.language-julia-1.79.2\scripts\pack

In [ ]:
trapezoidal(x -> x, 0,1,1)

In [ ]:
trapezoidal(x -> 1, 0,1,10)

Our function misbehaves when the number of subintervals is zero negative; we should fix that.

In [ ]:
trapezoidal(x -> 1, 0,1, -9)

In [ ]:
trapezoidal(x -> 1, 0,1, 0)

Another misfeature, I think, is that our function will not do exact rational arithmetic. That's because the way `h` is evaluated is converted to a Float64. We can fix this by extending the operator // to work for any float divided by an integer.

In [ ]:
import Base.//

In [ ]:
//(x::T, y::Integer) where {T<:AbstractFloat} = x / y

In [ ]:
"""
    trapezoidial(f::Function, a::Number, b::Number, n::Integer)

Return the n-panel trapezoidal rule estimation for `integrate(f(x),x,a,b)`
"""
function trapezoidal(f::Function, a::Number, b::Number, n::Integer)
    n > 0 || throw(ArgumentError("The number of panels must be positive, found $n"))
    h = (b-a)//n
    s = (f(a) + f(b))//2
    for k = 1 : n-1
        s += f(fma(h,k,a))
    end
    h*s
end

In [ ]:
trapezoidal(x -> x, 0, 1, -10)

In [ ]:
trapezoidal(x -> BigInt(1)//x, BigInt(1)//1, BigInt(2)//1, 10)

In [ ]:
Here is a fun divide and conquer method for the Trapezoidal rule.

In [ ]:
function trapezoidal_r(f::Function, a::Number, b::Number, n::Integer)
  if n==1
        (b-a)*(f(a)+f(b))//2
  else
        mid = (a+b)//2
        n = div(n,2) + if isodd(n) 1 else 0 end
        trapezoidal_r(f,a,mid,n) + trapezoidal_r(f,mid,b,n) 
  end
end

Let's compare the divide and conquer method with the iterative method for `integrate(x *sin(x^2),x,0,1000)`. For the number of panels, we'll use a power of 2. That should make the knots for each method identical.

In [ ]:
(f, a, b, n) = (x -> x*sin(x^2), 0.0, 5.0, 2^28);

In [ ]:
Tr = trapezoidal_r(f,a,b,n)

In [ ]:
Ts = trapezoidal(f,a,b,n)

In [ ]:
The exact value of this integral is

In [ ]:
Texact = 1/2-cos(25)/2

In [ ]:
Which method gives the most accurate estimation?

In [ ]:
Tr - Texact

In [ ]:
Ts - Texact

In [ ]:
abs(Tr - Texact) < abs(Ts - Texact)

Why does the divide and conquer method have greater accuracy? It's because the divide and conquer method does a pairwise sum, and the pairwise sum generally has lower rounding error than a serial sum.


Here is an adaptive trapezoidal rule function. 

In [ ]:
"""
    Trapezoidial_adaptive(f::Function, a::Number, b::Number, maxError::Number,maxLength::Number)

Use and adaptive trapezoidal rule for `integrate(f(x),x,a,b)`. The method attempts to return a value
whose error is less than `maxError`. The largest step size is `maxLength`
"""
function trapezoidal_adaptive(f::Function, a::Number, b::Number, max_error::Number, max_length::Number)
    n = convert(Int, ceil((b-a)/max_length))
    i1 = trapezoidal(f,a,b,n)
    i2 = trapezoidal(f,a,b,2*n)
    e =  abs(4*(i2 - i1)/3)
    if e < max_error
        (4*i2 - i1)/3
    else
        mid = (a+b)//2
        trapezoidal_adaptive(f,a,mid, max_error/2, max_length) + trapezoidal_adaptive(f,mid, b,max_error/2, max_length) 
    end 
end

Our code is inefficient--there is redundancy in computing both `trapezoidal(f,a,b,n)` and `trapezoidal(f,a,b,2n)`. Maybe you all can fix that.

In [ ]:
 Ta = trapezoidal_adaptive(x -> x*sin(x^2),0,5, 1.0e-10, 0.1)

In [ ]:
Texact - Ta

Here is a function that can be problematic for numerical integration. It is very nearly zero everywhere except at a big spike near an endpoint. 

In [ ]:
f = x -> 1000001 * x^(1000000)

Its graph is misleading

In [ ]:
using Gadfly

In [ ]:
plot(f,0,1)

In [ ]:
trapezoidal(f,0.0,1.0,10^8)

In [ ]:
 trapezoidal_adaptive(f,0.0,1.0,1.0e-8,1)

Can we do better than the best? If not, we are losers:

In [ ]:
using QuadGK

In [ ]:
quadgk(f, 0, 1, rtol=1.0e-8)